Script para aumentar los datos de entrenamiento de un modelo de detección de objetos con la librería Albumentations
author: Alvaro Zambrana Sejas

In [1]:
# instalar librerías
!pip install opencv-python scikit-image albumentations

In [2]:
!python --version

Python 3.10.15


In [3]:
# Aumento de datos con la librería Albumentations

# instalar librerías
!pip install -U albumentations
#!pip install albumentations=


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!dir

 Volume in drive C has no label.
 Volume Serial Number is 860A-E6B1

 Directory of C:\Users\Alvaro\PycharmProjects\copilotoVirtual\notebooks

25/11/2024  10:57 p. m.    <DIR>          .
28/10/2024  12:00 p. m.    <DIR>          ..
04/10/2024  11:11 a. m.               588 4.5
26/10/2024  12:17 a. m.           145,381 analyze_timestamp.ipynb
01/10/2024  07:38 p. m.           805,169 aumento_de_datos-bup.ipynb
25/11/2024  10:57 p. m.            22,837 aumento_de_datos.ipynb
04/10/2024  09:42 a. m.         3,932,288 calibration_image_sample_data_20x128x128x3_float32.npy
08/10/2024  06:05 p. m.           284,445 copiloto_virtual_logs.ipynb
18/09/2024  01:45 a. m.    <DIR>          data
03/10/2024  12:54 a. m.    <DIR>          dataset
25/09/2024  08:21 a. m.    <DIR>          datasets
15/10/2024  11:45 p. m.               649 EDA.ipynb
27/09/2024  11:25 p. m.            73,752 ETL.ipynb
24/09/2024  12:05 p. m.             2,912 ETL2.ipynb
27/10/2024  11:51 a. m.         2,469,057 extract_m

In [1]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os
from skimage import io
import cv2
 
# cargar las imágenes
ruta = '../dataset/processed/yolo_signals_cbba/train'
ruta_salida = '../dataset/processed/yolo_signals_cbba_with_augmented_data/train'
jpg_extension = '.jpg'
txt_extension = '.txt'

# Definimos la función para cargar las imágenes
def cargar_imagenes(ruta, extension):
    imagenes = {}    
    
    for archivo in os.listdir(ruta):
        path, ext = os.path.splitext(archivo)
        basename = os.path.basename(path)
        if archivo.endswith(extension):
            # imagen = io.imread(os.path.join(ruta, archivo))
            image = cv2.imread(os.path.join(ruta, archivo))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # crear un diccionario con la imagen y el basename como key
            imagenes[basename] = image
    return imagenes


# Definimos la función para cargar las imágenes
def cargar_bboxes(ruta, ruta_images, extension):
    imagenes_bboxes = {}
    for archivo in os.listdir(ruta_images):
        path, ext = os.path.splitext(archivo)
        basename = os.path.basename(path)
        txt_file = f'{basename}{extension}'
        print(txt_file, basename)
        
        if os.path.exists(os.path.join(ruta, txt_file)):
            txt = open(os.path.join(ruta, txt_file), 'r')
            bboxes = []
            for line in txt:
                bbox = line.split()
                bboxes.append(bbox)

            imagenes_bboxes[basename] = bboxes

    return imagenes_bboxes

# Configuración de las transformaciones
transformaciones = [
    A.RandomBrightnessContrast(p=0.15),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=10, val_shift_limit=10, p=0.5),
    
    A.RandomScale(scale_limit=0.1, p=0.25),
    A.Rotate(limit=(-15, 15), p=0.2),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=5, p=0.15),
    A.Perspective(scale=(0.05, 0.05), p=0.3),    
    A.Affine(scale=0.9,translate_percent=0.05,shear=(-5,5),p=0.1),

    A.Downscale(always_apply=False, p=0.3, scale_min=0.35, scale_max=0.79),
    A.MotionBlur(always_apply=False, p=0.3, blur_limit=(15, 31), allow_shifted=True)
]

# Definir las transformaciones
transform = A.Compose(transformaciones, bbox_params= A.BboxParams(format='yolo', min_visibility=0.8))
num_variaciones = 5  # Definir cuántas veces se quiere transformar la imagen aleatoriamente

# Definimos la función para aplicar las transformaciones de los datos con Albumentations
def aumentar_datos_albumentations(imagenes, bboxes):
    imagenes_aumentadas = []
    datos_aumentados = {}
    for basename, imagen in imagenes.items():
        print(f'Imagen original de {basename}')
        print(f'bbox original de {basename}', bboxes[basename])
        # Aplicar las transformaciones varias veces 
        for i in range(num_variaciones):
            # convertir las bboxes a formato de Albumentations
            image_bboxes = []
            for bbox in bboxes[basename]:
                image_bboxes.append([float(bbox[1]), float(bbox[2]), float(bbox[3]), float(bbox[4]), int(bbox[0])])               

            try:
                imagen_aumentada = transform(image=imagen, bboxes = image_bboxes)
                print(f'{basename}_{i}', imagen_aumentada['image'], imagen_aumentada['bboxes'])
                imagenes_aumentadas.append(imagen_aumentada['image'])
                print(f'Imagen aumentada de {basename}')
                print(f'bbox aumentada de {basename}', image_bboxes)
                datos_aumentados[f'{basename}_{i}'] = imagen_aumentada
            except Exception as e:
                print(f'Error en la imagen {basename}_{i}: {e}')
                continue

    return datos_aumentados

imagenes = cargar_imagenes(f'{ruta}/images', jpg_extension)
bboxes = cargar_bboxes(f'{ruta}/labels', f'{ruta}/images', txt_extension)

# Aumentar los datos con Albumentations
imagenes_aumentadas_albumentations = aumentar_datos_albumentations(imagenes, bboxes) 

# almacenar las imágenes y etiquetas aumentadas
def guardar_imagenes_y_etiquetas(datos_aumentados, ruta_salida):
    
    if not os.path.exists(ruta_salida):
        os.makedirs(ruta_salida)
    
    for basename, imagen_aumentada in datos_aumentados.items():
        imagen = imagen_aumentada['image']
        bboxes = imagen_aumentada['bboxes']

        io.imsave(os.path.join(f'{ruta_salida}/images', f'{basename}_transformado.jpg'), imagen)

        with open(os.path.join(f'{ruta_salida}/labels', f'{basename}_transformado.txt'), 'w') as file:
            for bbox in bboxes:
                file.write(f'{bbox[4]} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n')
    return  

guardar_imagenes_y_etiquetas(imagenes_aumentadas_albumentations, ruta_salida)


KeyboardInterrupt: 

In [19]:
# render images with bounding boxes

import cv2
import matplotlib.pyplot as plt
import numpy as np

def render_image_with_bboxes(image, bboxes):
    for bbox in bboxes:
        x, y, w, h, _ = bbox
        # denormalize the bounding box
        x, y, w, h = int(x * image.shape[1]), int(y * image.shape[0]), int(w * image.shape[1]), int(h * image.shape[0])

        x1, y1, x2, y2 = int(x - w/2), int(y - h/2), int(x + w/2), int(y + h/2)
        image = cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
    return image

def render_images_with_bboxes(images, bboxes):
    fig, axs = plt.subplots(3, 3, figsize=(15, 15))
    for i, (image, bbox) in enumerate(zip(images, bboxes)):
        image_with_bboxes = render_image_with_bboxes(image, bbox)
        axs[i // 3, i % 3].imshow(image_with_bboxes)
        axs[i // 3, i % 3].axis('off')
    plt.show()
    return

def guardar_imagenes_con_sus_etiquetas(datos_aumentados, ruta_salida):
    for basename, datos in datos_aumentados.items():
        imagen = datos['image']
        bboxes = datos['bboxes']

        image_with_bboxes = render_image_with_bboxes(imagen, bboxes)

        io.imsave(os.path.join(ruta_salida, f'{basename}_etiquetada.jpg'), image_with_bboxes)        
    return  

guardar_imagenes_con_sus_etiquetas(imagenes_aumentadas_albumentations, ruta_salida)